In [2]:
!python -V

Python 3.11.9


### Import packages 

In [3]:
import pandas as pd

# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Package: pickle
# Purpose: This package provides functionality for working with pickle files.
# Pickle is a Python module used for serializing and deserializing Python objects.
# It allows you to save and load Python objects to and from disk, preserving their state.

import pickle
# Package: seaborn
# Purpose: Seaborn is a data visualization library based on Matplotlib.
# It provides a high-level interface for creating informative and attractive statistical graphics.
import seaborn as sns

# Package: matplotlib.pyplot
# Purpose: Matplotlib is a plotting library for creating static, animated, and interactive visualizations in Python. 
# pyplot is a collection of functions that provide a simple interface for creating plots and visualizations.
import matplotlib.pyplot as plt
# Package: DictVectorizer 
# Purpose: Convert a collection of dictionaries into a matrix representation
from sklearn.feature_extraction import DictVectorizer  

# Package: LinearRegression
# Purpose: Implement linear regression models
from sklearn.linear_model import LinearRegression  

# Package: Lasso
# Purpose: Implement Lasso regression models
from sklearn.linear_model import Lasso  

# Package: Ridge
# Purpose: Implement Ridge regression models
from sklearn.linear_model import Ridge  

# Package: mean_squared_error
# Purpose: Calculate the mean squared error between two arrays
from sklearn.metrics import mean_squared_error  
# Package: mlflow
# Purpose: MLflow is an open-source platform for managing the end-to-end machine learning lifecycle.

import mlflow
# Package: pyspark
# Purpose: PySpark is the Python API for Apache Spark. Apache Spark is a fast and general-purpose cluster computing system.

import pyspark

#Package: xgboost
#Purpose: XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable.

import xgboost as xgb

#Package: hyperopt
#Purpose: Hyperopt is a Python library for serial and parallel optimization over awkward search spaces, which may include real-valued, discrete, and conditional dimensions.

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import SparkTrials
from hyperopt.pyll import scope

### Preparation

- CD into ~/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/homework/
- Create new notebbook "homework_module_2"
- Create Data folder 
- CD into data folder
- Download Jan, Feb, March 2023 Green Taxi data 

In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

--2024-05-25 14:25:19--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.247.148, 3.164.247.127, 3.164.247.101, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.247.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘green_tripdata_2023-01.parquet’

green_tripdata_2023 100%[===================>]   1.36M  --.-KB/s    in 0.1s    

2024-05-25 14:25:19 (10.7 MB/s) - ‘green_tripdata_2023-01.parquet’ saved [1427002/1427002]

--2024-05-25 14:25:20--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.247.182, 3.164.247.101, 3.164.247.127, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.247.182|:443... connected.
HTTP request sen

### Question 1

In [3]:
!mlflow --version

mlflow, version 2.13.0


### Question 2

In [8]:
!python preprocess_data.py --raw_data_path ~/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/homework/data --dest_path ./output

/bin/bash: line 1: cd: ./cohorts/2024/02-experiment-tracking/homework: No such file or directory
Traceback (most recent call last):
  File "/home/ubuntu/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py", line 83, in <module>
    run_data_prep()
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/click/core.py", line 783, in invoke
    return __callbac